This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
response = requests.get(url, params={'start_date': '2023-05-17','end_date': '2023-05-17'})
response.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2023-05-17',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [],
  'collapse': None,
  'order': None}}

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#Set parameters to include all dates in 2017
params = {'start_date': '2017-01-01','end_date': '2017-12-31'}

response = requests.get(url, params=params)

#Get json object 
afx_2017 = response.json()

In [6]:
#Empty list to store all opening values from 2017
opening_values = []

#Access the data points
data = afx_2017['dataset_data']['data']

In [7]:
# Iterate through the data points
for item in data:
    # Access the 'Open' value
    open_val = item[1]
    opening_values.append(open_val)

# Print the first five opening values
print(opening_values[:5])

[51.76, 51.65, 51.45, 51.05, 51.16]


In [8]:
#Remove None values from the opening values list
valid_opening_values = list(filter(lambda x: x is not None, opening_values))

max_open = max(valid_opening_values)
min_open = min(valid_opening_values)

#Print the max and min opening values for AFX_X in 2017
print("Maximum opening value:", max_open)
print("Minimum opening value:", min_open)

Maximum opening value: 53.11
Minimum opening value: 34.0


In [9]:
daily_range = []

#Iterate through data to find daily range and append to daily range list
for item in data:
    high = item[2]
    low = item[3]
    difference = high-low
    daily_range.append(difference)
    
max_difference = round(max(daily_range), 2)
print("The largest change in any one day:", max_difference)

The largest change in any one day: 2.81


In [10]:
closing_prices = []

# Iterate through the data points
for item in data:
    # Access the 'Close' value
    closing_val = item[4]
    closing_prices.append(closing_val)

num_days = len(closing_prices)
largest_change = 0
for i in range(num_days - 1):
    for j in range(i + 1, num_days):
        difference = abs(closing_prices[i] - closing_prices[j])
        largest_change = max(largest_change, difference) 
        
print("The largest change between two consecutive days:",largest_change)

The largest change between two consecutive days: 19.03


In [11]:
traded_volume = []
for item in data:
    # Access the  value
    trading_vol = item[6]
    traded_volume.append(trading_vol)
    
average = sum(traded_volume) / len(traded_volume)    
print("The average daily trading volume:", average)

median_index = int((len(traded_volume)+ 1) / 2)
sorted_volume = sorted(traded_volume)
median_value = sorted_volume[median_index]
print("The median daily trading volume:", median_value)

The average daily trading volume: 89124.33725490196
The median daily trading volume: 76600.0
